In [0]:
%pip install yfinance 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 60.0 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=c67f167d75ed8ee158e474093c248fb85493287cb86dc2918089a1619fa4c016
  Stored in directory: /home/spark-7ed06652-b059-4107-bb78-4d/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.3-cp312-cp312-linux_x86_64.whl size=942261 sha256=f2556d63ad

In [0]:
import yfinance as yf
import pandas as pd
import numpy as np

In [0]:
# Configure authentication 
spark.conf.set("fs.azure.account.auth.type.stocksdl.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.stocksdl.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.stocksdl.dfs.core.windows.net", "CLIENT-ID")
spark.conf.set("fs.azure.account.oauth2.client.secret.stocksdl.dfs.core.windows.net", "SECRET-KEY")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.stocksdl.dfs.core.windows.net", "https://login.microsoftonline.com/TENANT-ID/oauth2/token")

In [0]:
# Importing Stock Tickers
ticker_aapl = "AAPL"
tic_aapl = yf.Ticker(ticker_aapl)

ticker_mrna = "MRNA"
tic_mrna = yf.Ticker(ticker_mrna)

ticker_nvda = "NVDA"
tic_nvda = yf.Ticker(ticker_nvda)

ticker_ocg = "OCG"
tic_ocg = yf.Ticker(ticker_ocg)

ticker_tsla = "TSLA"
tic_tsla = yf.Ticker(ticker_tsla)

In [0]:
# Reading Per Day Data
todays_aapl = tic_aapl.history(period="1d",interval="1m")
todays_aapl.reset_index(inplace=True)

todays_mrna = tic_mrna.history(period="1d",interval="1m")
todays_mrna.reset_index(inplace=True)

todays_nvda = tic_nvda.history(period="1d",interval="1m")
todays_nvda.reset_index(inplace=True)

todays_ocg = tic_ocg.history(period="1d",interval="1m")
todays_ocg.reset_index(inplace=True)

todays_tsla = tic_tsla.history(period="1d",interval="1m")
todays_tsla.reset_index(inplace=True)

In [0]:
# Converting to Spark and Dropping unnecessary cols for 'TODAYS DATA'
today_spark_aapl = spark.createDataFrame(todays_aapl)
today_spark_aapl = today_spark_aapl.drop('Stock Splits', 'Dividends')

today_spark_mrna = spark.createDataFrame(todays_mrna)
today_spark_mrna = today_spark_mrna.drop('Stock Splits', 'Dividends')

today_spark_nvda = spark.createDataFrame(todays_nvda)
today_spark_nvda = today_spark_nvda.drop('Stock Splits', 'Dividends')

today_spark_ocg = spark.createDataFrame(todays_ocg)
today_spark_ocg = today_spark_ocg.drop('Stock Splits', 'Dividends')

today_spark_tsla = spark.createDataFrame(todays_tsla)
today_spark_tsla = today_spark_tsla.drop('Stock Splits', 'Dividends')

In [0]:
# # First writing in delta format for daily data
# today_spark_aapl.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Apple/") # todays

# today_spark_mrna.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Moderna") # todays 
 
# today_spark_nvda.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Nvidia/") # todays

# today_spark_ocg.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Oriental/") # todays

# today_spark_tsla.write.mode("overwrite").format("delta").save("abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Tesla/") # todays

In [0]:
# Regular writing Daily Data
from delta.tables import DeltaTable


#Apple
deltaTable_aapl = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Apple/")
deltaTable_aapl.alias("target").merge(
    today_spark_aapl.alias("source"),
   "target.Datetime = source.Datetime"
).whenNotMatchedInsertAll().execute()

#Moderna
deltaTable_mrna = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Moderna/")
deltaTable_mrna.alias("target").merge(
    today_spark_mrna.alias("source"),
   "target.Datetime = source.Datetime"
).whenNotMatchedInsertAll().execute()

#Nvidia
deltaTable_nvda = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Nvidia/")
deltaTable_nvda.alias("target").merge(
    today_spark_nvda.alias("source"),
   "target.Datetime = source.Datetime"
).whenNotMatchedInsertAll().execute()

#Oriental
deltaTable_ocg = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Oriental/")
deltaTable_ocg.alias("target").merge(
    today_spark_ocg.alias("source"),
   "target.Datetime = source.Datetime"
).whenNotMatchedInsertAll().execute()

#Tesla
deltaTable_tsla = DeltaTable.forPath(spark, "abfss://bronze@stocksdl.dfs.core.windows.net/todays-data/Tesla/")
deltaTable_tsla.alias("target").merge(
    today_spark_tsla.alias("source"),
   "target.Datetime = source.Datetime"
).whenNotMatchedInsertAll().execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]